<h1>imports</h1>

In [1]:
# download (snippet from: https://colab.research.google.com/drive/1_Y-sZ5eHIDlDUMuLCwfnbuJdLh0DTXmO#scrollTo=5HGlaJTEAYJu&line=23&uniqifier=1)
import os

# We define the datasets we want to load
datasets = ('accounts', 'users', 'subscriptions', 'events')
source_prefix = 'https://storage.googleapis.com/mondaycom-datahack/final_sets/'

local_dir = './data/'
file_prefixs = ['train_','test_']
file_suffix = ''
file_extension = 'csv'

# We create a directory for the datasets if it doesn't exist
if not os.path.exists(local_dir):
    os.makedirs(local_dir)

for file_prefix in file_prefixs:
    # For each dataset we want, we check if we already downloaded it and fix it if we didn't
    for dataset in datasets:
      if not os.path.isfile('{}{}{}{}.{}'.format(local_dir, file_prefix, dataset, file_suffix, file_extension)):
        !curl {source_prefix}{file_prefix}{dataset}{file_suffix}.{file_extension} --output {local_dir}{file_prefix}{dataset}{file_suffix}.{file_extension}

In [2]:
import numpy as np
# import pandas as pd
import modin.pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.2f}'.format
import matplotlib.pyplot as plt
from urllib import request
import json
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, Normalizer, Binarizer
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# from imblearn.over_sampling import SMOTE
from collections import Counter

import time
# print each command: 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from IPython.core.debugger import set_trace

# from multiprocessing import  Pool

<h1>load data</h1>

In [3]:
data_path = './data/'

In [4]:
# %%time
train_accounts = pd.read_csv(data_path + 'train_accounts.csv')#.sample(frac=0.2)
train_users = pd.read_csv(data_path + 'train_users.csv')
train_events = pd.read_csv(data_path + 'train_events.csv')
train_subscriptions = pd.read_csv(data_path + 'train_subscriptions.csv')
test_accounts = pd.read_csv(data_path + 'test_accounts.csv')
test_users = pd.read_csv(data_path + 'test_users.csv')
test_events = pd.read_csv(data_path + 'test_events.csv')
test_subscriptions = pd.read_csv(data_path + 'test_subscriptions.csv')

KeyboardInterrupt: 

In [ ]:
%%time
accounts = pd.concat([train_accounts, test_accounts],sort=False)
print(f'accounts: {len(accounts)}')
users = pd.concat([train_users, test_users],sort=False)
print(f'users: {len(users)}')
events = pd.concat([train_events, test_events],sort=False)
print(f'events: {len(events)}')
subscriptions = pd.concat([train_subscriptions, test_subscriptions],sort=False)
print(f'subscriptions: {len(subscriptions)}')

In [ ]:
# convert to dask df: 
# accountsd=dd.from_pandas(accounts,npartitions=1)
# usersd=dd.from_pandas(users,npartitions=1)
# eventsd=dd.from_pandas(events,npartitions=1)
# subscriptions=dd.from_pandas(subscriptions,npartitions=1)
# dask.compute()

<h1>feature engineering</h1>

## direct numerical correlation
First lets exampain the correlation between all numerical columns in "accounts" and the target columns "lead_score":

In [ ]:
corrmat = accounts.select_dtypes('number').corr()
f, ax = plt.subplots(figsize = (11,9))
corrmat['lead_score'][0:-1].plot(kind='bar',rot=60)
plt.title('Correlation with lead_score')
plt.grid();

seems like the best features would be "company_size" and "mmr", and maybe also "paying", "collection_21_days" and "time_diff". 

Next, we examine numerical features from the other 3 tables. The approach here is to calculate the sum (and other several statistics) of each columns for each account, and then find which of them has correlation with the target: 

In [ ]:
def groupby_above(df,df_name,d):
    n_df = df.groupby('account_id')[df.columns[1]].count()
    idx1 = n_df.iloc[n_df.values<=d]
    idx2 = n_df.iloc[n_df.values>d]
    print('{} accounts with more than {} {}.'.format(len(idx2),d,df_name))
    return idx1, idx2
idx1_us, idx2_us = groupby_above(events,'events',500)


In [ ]:
%%time
# # tnames = ['users','events','subscriptions']
# # for i,table in enumerate([users,events,subscriptions]):

def extract_num_feat(df):
    cols=df.select_dtypes('number').columns # columns names
    n_df1 = accounts.set_index('account_id').loc[:,['lead_score']] #new dataframe with account as index and columns lead_score
    n_df2 = df.groupby('account_id').filter(lambda x: x[df.columns[1]].count()>500).groupby('account_id')[cols].describe()
    n_df = n_df2.join(n_df1)
#     return n_df,n_df1,n_df2
    return n_df

df_us = extract_num_feat(events)
# len(idx1_us)+len(idx2_us)
df_us.to_csv('accounts_above500events_feat.csv')
df_us

In [ ]:
# test_users.iloc[(test_users.groupby('account_id')['user_id'].count()>1).values]
# test_users.where
gbidx=test_users.groupby('account_id').filter(lambda x: x['user_id'].count()>10).groupby('account_id').count()
gbidx

In [ ]:
df = pd.DataFrame({'A':np.random.randint(-10,10,1000000),'B':np.random.random(1000000)})
df1 = df.groupby('A').filter(lambda x: x['A'].mean()>0).groupby('A').count()
df1
df2 = df[df.A >= 0].groupby('A').count()
df2

In [ ]:
a=test_users.groupby('account_id')
# b=a.iloc[idx2_us].count()

In [ ]:
x1=(lambda x: x['user_id'].count()>10)
# x1
df= test_users
# x1=(df.groupby('account_id')[df.columns[1]].count()>1)
# x1.values
a.filter(x1).count()

In [ ]:
# a.nth(list(idx2_us.index)).count()
a.iloc[x1.values].count()

In [ ]:
df_us

Note that there are more accounts in 'events' than in 'accounts': 

In [ ]:
events['account_id'].max()
accounts['account_id'].max()

In [ ]:
t_name

In [ ]:
def extract_num_feat(df):
    cols=df.select_dtypes('number').columns # columns names
    n_df1 = accounts.set_index('account_id').loc[:,['lead_score']] #new dataframe with account as index and columns lead_score
    #     for col in cols: #create features for each column
    # col_names = [col+'_'+stat for i,col in enumerate(cols)]
    n_df2 = df[cols].describe()
    n_df = n_df2.join(n_df1)
#     return n_df,n_df1,n_df2
    return n_df
       
st=time.time()
# ex_feat_ev,ev1,ev2 = extract_num_feat(events.iloc[idx_ev])
# print(time.time()-st)
ex_feat_us,us1,us2 = extract_num_feat(users.iloc[idx_us])
print(time.time()-st)
# # ex_feat_sb,sb1,sb2 = extract_num_feat(subscriptions.iloc[0:1000])
# print(time.time()-st)

In [ ]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
%%time
ex_feat_us,us1,us2=parallelize_dataframe(users.iloc[0:10000], extract_num_feat, n_cores=4)

In [ ]:
%%time
for ex_feat in list([ex_feat_ev,ex_feat_us]):
    corrs = ex_feat.select_dtypes('number').corrwith(ex_feat.lead_score)
    f, ax = plt.subplots(figsize = (11,9))
    print(corrs[corrs.abs().values>0.06][:-1])
    corrs[corrs.abs().values>0.07][:-1].plot(kind='bar',rot=60)
    plt.title('Correlation with lead_score')
    plt.grid();

In [ ]:
corrs[corrs.abs().values>0.05][:-1]

look at
churn date
user role
domain

In [ ]:
# transform plan_id & utm_cluster_id to str since its categorical
accounts['plan_id'] = accounts['plan_id'].astype(str)
accounts['utm_cluster_id'] = accounts['utm_cluster_id'].astype(str)

In [ ]:
def clip_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_in.loc[df_in[col_name] > fence_high, col_name] = fence_high
    df_in.loc[df_in[col_name] < fence_low, col_name] = fence_low
    
def creating_time_features(data):
    time_between_created_trial = pd.to_datetime(data['trial_start']) - pd.to_datetime(data['created_at'])
    time_between_created_subscription = pd.to_datetime(data['subscription_started_at']) - pd.to_datetime(data['created_at'])
    time_between_trial_subscription = pd.to_datetime(data['subscription_started_at']) - pd.to_datetime(data['trial_start'])
    time_between_now_trial = datetime.now() - pd.to_datetime(data['trial_start'])
    time_between_now_subscription = datetime.now() - pd.to_datetime(data['subscription_started_at'])
    time_between_now_created = datetime.now() - pd.to_datetime(data['created_at'])
    time_between_now_churn = datetime.now() - pd.to_datetime(data['churn_date'])
    time_between_churn_subscription = pd.to_datetime(data['churn_date']) - pd.to_datetime(data['subscription_started_at'])

    data = data.assign(created_trial_delta=time_between_created_trial.apply(lambda x: (x.seconds//3600)))
    data = data.assign(created_subscription_delta=time_between_created_subscription.apply(lambda x: (x.seconds//3600)))
    data = data.assign(trial_subscription_delta=time_between_trial_subscription.apply(lambda x: (x.seconds//3600)))
    data = data.assign(now_trial_delta=time_between_now_trial.apply(lambda x: (x.seconds//3600)))
    data = data.assign(now_subscription_delta=time_between_now_subscription.apply(lambda x: (x.seconds//3600)))
    data = data.assign(now_created_delta=time_between_now_created.apply(lambda x: (x.seconds//3600)))
    data = data.assign(now_churn_delta=time_between_now_churn.apply(lambda x: (x.seconds//3600)))
    data = data.assign(churn_subscription_delta=time_between_churn_subscription.apply(lambda x: (x.seconds//3600)))
    data['is_subscription'] = (data.subscription_started_at.isna()).astype(int)
    data['is_churn'] = (data.churn_date.isna()).astype(int)
    return data

def creating_size_n_survey_features(data, bins, bins_labels):
    #clip_outlier(data,'company_size')
    data.loc[:,'avg_team_size'] = data[["min_team_size", "max_team_size"]].mean(axis=1)
    data['avg_team_size'].fillna(-1, inplace=True)
    data['avg_team_cat'] = pd.cut(data['avg_team_size'], bins=bins, labels=bins_labels)
    data['avg_team_cat'] = data['avg_team_cat'].astype(str)
    data['survey_answers'] = data[['company_size','max_team_size','min_team_size','user_goal','user_description','team_size']].isna().sum(axis=1)
    data['survey_did_answer'] = data['survey_answers']
    return data

In [ ]:
# creating size & survey features
bins = sorted((list(train_accounts["max_team_size"].value_counts().index) + [-1.1, -1, ]))
bins_labels = [str(b) for b in bins[1:]]

accounts = creating_time_features(accounts)
accounts = creating_size_n_survey_features(accounts, bins, bins_labels)
accounts['country_counts'] = accounts.groupby('country')['country'].transform('count')
accounts['region_counts'] = accounts.groupby('region')['region'].transform('count')

<h1>preprocessing</h1>

In [ ]:
# We map our features into different types
categorical_features = ['os', 'browser', 'payment_currency', 'device', 'industry', 'utm_cluster_id',
                         'plan_id', 'avg_team_cat']
normalized_features = ['collection_21_days', 'mrr', 'created_trial_delta', 'created_subscription_delta',
                       'trial_subscription_delta', 'now_trial_delta', 'now_subscription_delta', 'now_created_delta', 
                       'now_churn_delta', 'churn_subscription_delta', 'company_size', 'survey_answers']

binary_features = ['survey_did_answer']
untouched_features = ['paying', 'is_subscription', 'is_churn']
KBinsDiscretized_features = []
target = ['lead_score']

# And create a column transformer to handle the manipulation for us
preprocess = make_column_transformer(
    (OneHotEncoder(), categorical_features),
    (Normalizer(), normalized_features),
    (Binarizer(), binary_features)
)

In [ ]:
if 'account_id' in accounts.columns:
    accounts.set_index('account_id', inplace=True)
    #test_accounts.set_index('account_id', inplace=True)

    # Filling empty values with default values 
def fill_empty_values(dataset):
    dataset.loc[:,categorical_features] = dataset[categorical_features].fillna('')
    dataset.loc[:,normalized_features + binary_features + untouched_features] = dataset[normalized_features + binary_features + untouched_features].fillna(0)
    return dataset

accounts = fill_empty_values(accounts)

In [ ]:
def under_sample(data, target, neg_pos_ratio=1):
    target_num = len(data[data[target] == 1])
    negative_idx = data[data[target] == 0].index
    positve_idx = data[data[target] == 1].index
    rnd_negative = np.random.choice(negative_idx , target_num * neg_pos_ratio, replace=False)
    under_sample_idx = np.concatenate([positve_idx, rnd_negative])
    return data.loc[under_sample_idx]

In [ ]:
# We fit our column transformer on both the train and the test sets
preprocess.fit(accounts.drop('lead_score',axis=1))

#dataset_train = accounts.loc[train_accounts.account_id]
dataset_train = under_sample(accounts.loc[train_accounts.account_id], 'lead_score', 20)
dataset_test = accounts.loc[test_accounts.account_id].drop('lead_score',axis=1)

# We use transform to finally manipulate the features of our training set
x = preprocess.transform(dataset_train.drop('lead_score',axis=1))
# Seperating the label
y = dataset_train['lead_score']
print(f'train data size {len(dataset_train)}')

In [ ]:
x.shape

<h1>train model</h1>

from sklearn.feature_selection import SelectFromModel

log_r = LogisticRegression(class_weight='balanced', penalty='l1', n_jobs=-1)
sfm = SelectFromModel(log_r, threshold=0.5)
x = sfm.fit_transform(x, y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=42)

#sm = SMOTE(random_state=42)
#x_train, y_train = sm.fit_resample(x_train, y_train)

model = LogisticRegression(class_weight='balanced', penalty='l1', n_jobs=-1) # 'penalty': ['l1', 'l2'], 'C': [1, 10, 100, 1000]
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=42)
st=time.time()
scores = ['f1','recall'] # 'precision', 'recall', 'f1', 
hyparam_grid = [{
    'n_estimators': [50,100,300],
    'criterion': ['entropy'],
    'max_depth': [5,10],
    'class_weight': ['balanced'],
}]

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(RandomForestClassifier(n_jobs=-1), hyparam_grid, cv=4,
                       scoring='%s_macro' % score, n_jobs=1)
#     clf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=5, class_weight='balanced')
    clf.fit(x_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
    print()
    print(time.time()-st)

In [ ]:
st=time.time()
time.time()-st

In [ ]:
print(classification_report(y_test, y_pred, target_names=['not lead','lead']))
print('Acc:  {}'.format(metrics.accuracy_score(y_test, y_pred)))
print('MCC: {}'.format(metrics.matthews_corrcoef(y_test, y_pred)))
print('F1:  {}'.format(metrics.f1_score(y_test, y_pred)))

<h1>submit</h1>

In [ ]:
# sfm.transform(x_submission)

In [ ]:
x_submission = preprocess.transform(dataset_test)
#x_submission = sfm.transform(x_submission)
clf.fit(x, y)
y_pred_submission = clf.predict(x_submission)
# Creating a dictionary where the keys are the account_ids
# and the values are your predictions
submission_account_ids = [str(int(i))for i in dataset_test.index]
predictions = dict(zip(submission_account_ids, map(int, y_pred_submission)))

In [ ]:
group_name = 'fRidaY'

In [ ]:
# We validate first that we actually send all the test accounts expected to be sent
if y_pred_submission.shape[0] != 71683 or len(submission_account_ids) != 71683:
  raise Exception("You have to send all of the accounts! Expected: (71683, 71683), Got: ({}, {})".format(y_pred_submission.shape[0], submission_account_ids.shape[0]))

if "group_name" not in vars() or group_name == "":
  group_name = input("Please enter your group's name:")

data = json.dumps({'submitter': group_name, 'predictions': predictions}).encode('utf-8')

req = request.Request("https://leaderboard.datahack.org.il/monday/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)

res = request.urlopen(req)
print(json.load(res))

In [ ]:
0.366940718269732
0.3697461136151904